In [1]:
# 导入DeepSeek API KEY
import os

os.environ["DEEPSEEK_API_KEY"]='sk-7700dcfd8aec4e099191eade0488b209'
#os.environ["DEEPSEEK_API_KEY"]='<YOUR_DEEPSEEK_API_KEY>'
os.environ["LLM_MODEL"]='deepseek-chat'

# 没有记忆的模型

In [7]:
from openai import OpenAI

def get_response(input):
    
    client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com")
    
    response = client.chat.completions.create(
        model="deepseek-chat", # DeekSeep模型ID, 可选值为[deepseek-chat, deepseek-reasoner]
        messages=[
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": input},
        ],
        stream=False,
        temperature=0, # 采样温度，介于 0 和 2 之间, 更低的值会使其更加集中和确定
        max_tokens=2000 # 限制一次请求中模型生成 completion 的最大 token 数, 默认值为4096
    )
    return response.choices[0].message.content

In [8]:
print(get_response("你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？"))
print(get_response("我一般周几健身？"))

你好！很高兴听到你坚持健身的习惯，这对保持健康非常有帮助。今天是周二，我在这里随时准备帮助你解答问题或提供信息。如果你有任何关于健身、饮食、健康或其他方面的问题，都可以随时问我！祝你今天的健身顺利，保持好状态！
您好，建议您开启定位权限，这样我就能根据您所在的位置提供更精准的信息了。如果您需要查询其他城市的健身场所，也可以告诉我，我会尽力为您提供帮助。


# 通过Gradio快速展示功能

In [10]:
pip install -i https://mirrors.aliyun.com/pypi/simple/ gradio

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 15.0 MB/s eta 0:00:00m eta 0:00:010:01:01
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 14.0 MB/s eta 0:00:00 MB/s eta 0:00:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 16.2 MB/s eta 0:00:0031m19.8 MB/s eta 0:00:01
  Created wheel for markupsafe: filename=MarkupSafe-2.1.5-cp313-cp313-macosx_10_13_universal2.whl size=18089 sha256=19c157295ab98aca3ce381469a14db4752f4f9be004f0b6f8f8c0a65e282e13f
  Stored in directory: /Users/yihui.li/Library/Caches/pip/wheels/dc/2c/81/0b897bfd6c55baee2a710fcf1add61983a6582c170b1aa3e71
Successfully built markupsafe
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2


In [11]:
import gradio as gr
def respond(message, chat_history):
        bot_message = get_response(message)
        chat_history.append((message, bot_message)) #保存历史对话记录，用于显示
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #对话框
    msg = gr.Textbox(label="Prompt") #输入框
    btn = gr.Button("Submit") #提交按钮
    #提交
    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) 
gr.close_all()
demo.launch()

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio/components/chatbot.py:284: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


# 提供外部记忆

In [13]:
from openai import OpenAI
def get_response(msg):

    client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com")
    
    response = client.chat.completions.create(
        model="deepseek-chat", # DeekSeep模型ID, 可选值为[deepseek-chat, deepseek-reasoner]
        messages=msg,
        stream=False,
        temperature=0.9, 
        max_tokens=2000
    )

    return response.choices[0].message.content

In [14]:

def history_to_prompt(chat_history): # 将对话内容保存在一个List里
    msg = [{"role": "system", "content": "You are an AI assistant."}]
    i = 0
    for round_trip in chat_history: # 将List里的内容，组成 ChatCompletion的 messages部分，{role，content} dict
        msg.append({"role": "user", "content": round_trip[0]})
        msg.append({"role": "assistant", "content": round_trip[1]})
    return msg

def respond(message, chat_history):
    his_msg = history_to_prompt(chat_history) #并装历史会话，ChatCompletion的 messages部分格式
    his_msg.append({"role": "user", "content": message}) # 放入当前用户问题
    bot_message = get_response(his_msg)
    chat_history.append((message, bot_message)) # 将用户问题和返回保存到 历史记录 List
    return "", chat_history



In [18]:
import gradio as gr
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=480) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit
gr.close_all()
demo.launch(share=True)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio/components/chatbot.py:284: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864

Could not create share link. Missing file: /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio/frpc_darwin_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_darwin_amd64
2. Rename the downloaded file to: frpc_darwin_amd64_v0.3
3. Move the file to this location: /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio


# LangChain记忆功能

In [23]:
from langchain_openai.chat_models.base import BaseChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

llm = BaseChatOpenAI(model=os.getenv("LLM_MODEL"), 
                     openai_api_key=os.getenv("DEEPSEEK_API_KEY"), 
                     openai_api_base='https://api.deepseek.com',
                     max_tokens=1024)

memory = ConversationBufferWindowMemory(k=10)

def get_response(input):
    print("------------")
    print(memory.load_memory_variables({}))
    print("------------")
    conversation_with_memory = ConversationChain(
        llm=llm, 
        memory=memory,
        verbose=False
    )
    return conversation_with_memory.predict(input=input)

In [24]:
print(get_response("你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？"))
print(get_response("我一般周几健身？"))

------------
{'history': ''}
------------
你好！很高兴听到你每周二都有健身的习惯，这真是保持健康的好方法！至于我，作为一个AI，我没有具体的日程安排，但我的“任务”就是随时准备好回答你的问题、提供帮助或陪你聊天。如果你有任何关于健身、饮食、或者其他方面的问题，我都很乐意帮忙！你今天打算做哪些健身项目呢？
------------
{'history': 'Human: 你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？\nAI: 你好！很高兴听到你每周二都有健身的习惯，这真是保持健康的好方法！至于我，作为一个AI，我没有具体的日程安排，但我的“任务”就是随时准备好回答你的问题、提供帮助或陪你聊天。如果你有任何关于健身、饮食、或者其他方面的问题，我都很乐意帮忙！你今天打算做哪些健身项目呢？'}
------------
你刚才提到你一般每周二去健身，所以按照你提供的信息，你通常是在周二健身。如果你还有其他健身安排或者想要调整健身计划，我可以帮你提供一些建议或参考哦！😊


In [25]:
import gradio as gr
def respond(message, chat_history):
    bot_message = get_response(message)
    chat_history.append((message, bot_message))
    return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=300) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit
gr.close_all()
demo.launch(share=True)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio/components/chatbot.py:284: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865

Could not create share link. Missing file: /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio/frpc_darwin_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_darwin_amd64
2. Rename the downloaded file to: frpc_darwin_amd64_v0.3
3. Move the file to this location: /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio


------------
{'history': 'Human: 你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？\nAI: 你好！很高兴听到你每周二都有健身的习惯，这真是保持健康的好方法！至于我，作为一个AI，我没有具体的日程安排，但我的“任务”就是随时准备好回答你的问题、提供帮助或陪你聊天。如果你有任何关于健身、饮食、或者其他方面的问题，我都很乐意帮忙！你今天打算做哪些健身项目呢？\nHuman: 我一般周几健身？\nAI: 你刚才提到你一般每周二去健身，所以按照你提供的信息，你通常是在周二健身。如果你还有其他健身安排或者想要调整健身计划，我可以帮你提供一些建议或参考哦！😊'}
------------
------------
{'history': 'Human: 你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？\nAI: 你好！很高兴听到你每周二都有健身的习惯，这真是保持健康的好方法！至于我，作为一个AI，我没有具体的日程安排，但我的“任务”就是随时准备好回答你的问题、提供帮助或陪你聊天。如果你有任何关于健身、饮食、或者其他方面的问题，我都很乐意帮忙！你今天打算做哪些健身项目呢？\nHuman: 我一般周几健身？\nAI: 你刚才提到你一般每周二去健身，所以按照你提供的信息，你通常是在周二健身。如果你还有其他健身安排或者想要调整健身计划，我可以帮你提供一些建议或参考哦！😊\nHuman: 我一般周二健身，你今天干什么？\nAI: 你好！很高兴再次和你聊天！你提到你一般每周二健身，这真是个很棒的习惯！至于我，作为一个AI，我没有具体的日程安排，但我随时都在这里陪你聊天或回答你的问题。如果你今天健身时有什么特别的目标或问题，比如想了解一些新的健身动作、饮食建议，或者如何提高健身效果，我都很乐意帮忙！你今天健身有什么特别的计划吗？😊'}
------------


# ConversationSummaryBufferMemory

In [30]:
from langchain_openai.chat_models.base import BaseChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain

llm = BaseChatOpenAI(model=os.getenv("LLM_MODEL"), 
                     openai_api_key=os.getenv("DEEPSEEK_API_KEY"), 
                     openai_api_base='https://api.deepseek.com',
                     max_tokens=1024)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=1024)

def get_response(input):
    print("------------")
    print(memory.load_memory_variables({}))
    print("------------")
    conversation_with_memory = ConversationChain(
        llm=llm, 
        memory=memory,
        verbose=False
    )
    return conversation_with_memory.predict(input=input)

In [32]:
import gradio as gr
def respond(message, chat_history):
        bot_message = get_response(message)
        chat_history.append((message, bot_message))
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #对话框
    msg = gr.Textbox(label="Prompt") #输入框
    btn = gr.Button("Submit") #提交按钮
    #提交
    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) 
gr.close_all()
demo.launch()

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio/components/chatbot.py:284: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
